

## <span style="color:#4375c7">DAI</span>
***
*Course materials are for educational purposes only. Nothing contained herein should be considered investment advice or an opinion regarding the suitability of any security. For more information about this course, please contact us.*
***

### Session contents:
1. **[Deep Learning - Generative models ](#NN)**
    - [Variational autoencoder](#VAE)
    - [Generative adversarial network](#GAN)

    
2. **[Hands-on session](#ho)**
***



<br/><br/>

## 2. Hands-on session <a id='ho'></a>

### Exercise 1
1. Read the day-ahead electricity data set and visualize a random set of daily auctions. Each auction contains 24 load auction prices, one for each hour of the day. Fill in the missing parts of the code marked with ___. 

2. Use the given structure of a GAN to generate new time series. Fill in the missing parts of the code marked with ___. Use dense layers instead of convolutional layers. Execute the code and generate new time series.


### Exercise 2
1.  Read the day-ahead electricity dataset and split the dataset into a test dataset and a train dataset. Do a split after four years of data. Keep leap years in mind. 

2. Use the given structure of a VAE to generate new time series. Fill in the missing parts of the code marked with ___. Use dense layers instead of convolutional layers. Execute the code and generate new time series.


### Solution exercise 1

In [ ]:
import os
os.getcwd()
os.chdir('/home/jovyan/work/DAI')  # Provide the new path here

In [ ]:
#1)
import pandas as ___
import numpy as ___
import matplotlib.___ as ___
#read in the data and plot a random set
#read in the data
x_train = pd.DataFrame(pd.read_csv('___.csv')[1:]).values

plt.plot(x_train[np.random.___(1,1000)])
plt.xlabel("hours")
plt.ylabel("dayahead price")
plt.show()


In [ ]:
#2)
# Lets start with the gan
#define your parameters

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tqdm import tqdm

latent_dim = 24
iterations = 10000
batch_size = 100

# Read in the data
shapes = pd.read_csv('Dataset_dayahead.csv')

# Convert the data to float32
x_train = np.array(shapes).astype(np.float32)

#define the generator
generator_input = keras.Input(shape=(latent_dim, ))
x = layers.___(24)(___)
x = layers.___(0.3)(x)
x = layers.___(128)(x)
x = layers.___(0.3)(x)
x = layers.___(256)(x)
x = layers.___(0.3)(x)
x = layers.___(24, activation='___')(x)
generator_output = keras.models.Model(generator_input, generator_output)
generator.summary()

In [ ]:
#define the discriminator
discriminator_input = layers.Input(shape=(latent_dim, ))
x = layers.___(256)(discriminator_input)
x = layers.___(0.3)(x)
x = layers.___(128)(x)
x = layers.___(0.3)(x)
x = layers.Dropout(0.3)(x)  #important trick for robustness
discriminator_output = layers.___(1, activation='___')(x)
discriminator = keras.models.Model(discriminator_input, discriminator_output)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(learning_rate=0.0004,
                                                   clipvalue=1.0,
                                                   decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
                      loss='___')

In [ ]:
#GAN model: discriminator not trainable during the training of the Generator !
discriminator.trainable = ___
gan_input = keras.Input(shape=(latent_dim, ))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
#gan.summary()

gan_optimizer = keras.optimizers.RMSprop(learning_rate=0.004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='___')


In [ ]:
loss_a, loss_d = [], []
start = 0

#train:
for step in tqdm(range(iterations)):
    # Draw random points from latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    # Generate artificial data points with the generator
    generated_ts = generator.predict(___)  

    # Mix real and fake data points
    stop = start + batch_size
    real_ts = x_train[start:stop]
    combined_ts = np.concatenate([generated_ts, real_ts])
    labels = np.concatenate(
        [np.___((batch_size, 1)),
         np.___((batch_size, 1))])
    # Add random noise to the labels for robustness
    labels += 0.05 * np.random.random(labels.shape)
    # Unfreeze the discriminator weights
    discriminator.trainable = ___
    #train the discriminator
    d_loss = discriminator.train_on_batch(combined_ts, labels)

    #draw new random points
    random_latent_vector = np.random.normal(size=(batch_size, latent_dim))
    #set misleading targets
    misleading_targets = np.zeros((batch_size, 1))
    #freeze the discriminator weights
    discriminator.trainable = ___

    #train the generator
    a_loss = gan.train_on_batch(random_latent_vectors,
                                ___)  #train the generator 

    #go to the next batch
    start += batch_size

    if start > len(x_train) - batch_size:
        start = 0
    # Add losses of generator and discriminator for later plots
    if step % 10 == 0:  # Record losses every 10 steps
        print(f'discriminator loss at step {step}: {d_loss}')
        print(f'adversarial loss at step {step}: {a_loss}')
        loss_a.append(___)
        loss_d.append(___)

In [ ]:
#plot the learning process
plt.plot(___, label="Loss generator")
plt.plot(___,label="loss discriminator")
plt.legend()
plt.ylabel('loss')
plt.xlabel('iterations')
plt.show()

In [ ]:
#generate new time series
random_latent_vectors = np.___.___(size=(1, latent_dim))
gen_ts = generator.predict(___)
plt.plot(pd.DataFrame(gen_ts).T)
plt.xlabel("hour")
plt.ylabel("dayahead energy price")
plt.show()

### Solution exercise 2

In [ ]:
#1) 
import pandas as ___
import numpy as ___
import keras
from keras import layers
from keras import backend as K
from keras.models import ___
import numpy as ___
import tensorflow as tf 
import matplotlib.___ as ___

#read in the data
shapes = pd.read_csv('Dataset_dayahead.csv')

#split the data in train and test datset and convert the data to float32
x_train = np.array(shapes.iloc[0:365 * ___ + ___, :]).astype(___.___)
x_test = np.array(shapes.iloc[365 * ___ - 1:365 * ___]).astype(___.___)

In [ ]:
#2)

#define the parameters of the vae
input_shape = (x_train.shape[1], )
latent_dim = 8
batch_size = 16
epochs = 50

#define the encoder
inputs = keras.Input(shape=input_shape)
x = layers.___(128, activation='___')(inputs)
x = layers.___(64, activation='___')(x)
x = layers.___(32, activation='___')(x)

z_mean = layers.Dense(___)(x)
z_log_var = layers.Dense(___)(x)


#reparametrization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim), mean=0., stddev=1.0)
    return z_mean + K.exp(z_log_var) * ___

# Use Lambda layer to include the sampling function in the model
z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name="encoder")

#define the encoder
# This is the input where we will feed `z`.
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.___(32, activation='___')(___)
x = layers.___(64, activation='___')(x)
x = layers.___(128, activation='___')(x)
x = layers.Dense(input_shape[0], activation='linear')(x)


# build the decoder in keras:
decoder = Model(decoder_input, x, name="decoder")

#decode the latent space variable
z_decoded = decoder(___)

In [ ]:
# Custom VAE model
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        self.add_loss(tf.reduce_mean(kl_loss) / tf.cast(tf.shape(inputs)[0], tf.float32))
        return reconstructed

vae = VAE(encoder, decoder)
vae.compile(optimizer='rmsprop', loss='mse')

In [ ]:
# Train the VAE
vae.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, x_test))

### References
***
[1] Keras online documentation: https://keras.io/about/

[2] Rashid, T. (2017). Neuronale Netze selbst programmieren: ein verständlicher Einstieg mit Python. O'Reilly.

[3] Nguyen, C. N., & Zeigermann, O. (2018). Machine Learning: kurz & gut.

[4] Chollet, F. (2018). Deep Learning mit Python und Keras: Das Praxis-Handbuch vom Entwickler der Keras-Bibliothek. MITP-Verlags GmbH & Co. KG. Code available on: https://github.com/fchollet/deep-learning-with-python-notebooks

[5]  Jason Brownlee (2019). https://machinelearningmastery.com/impressive-applications-of-generative-adversarial-networks/

[6] Kingma, D. P., & Welling, M. (2013). Auto-encoding variational bayes. arXiv preprint arXiv:1312.6114.

[7] Rezende, D. J., Mohamed, S., & Wierstra, D. (2014). Stochastic backpropagation and approximate inference in deep generative models. arXiv preprint arXiv:1401.4082.

[8] Goodfellow, I., Pouget-Abadie, J., Mirza, M., Xu, B., Warde-Farley, D., Ozair, S., Courville, A. & Bengio, Y. (2014). Generative adversarial nets. In Advances in neural information processing systems (pp. 2672-2680).